In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-matching/DATA.xlsx


In [5]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 106.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [34]:
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification,BertForSequenceClassification
from torch.optim import AdamW

import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [8]:
data_path='/kaggle/input/data-matching/DATA.xlsx'

In [9]:
data=pd.read_excel(data_path)

In [ ]:
data

In [ ]:
data = data[data['Description'].apply(lambda x: isinstance(x, str))]


In [ ]:
data = data.copy()

In [14]:

le = LabelEncoder()
data['label'] = le.fit_transform(data['Rep'])  # Now labels start from 0

# Confirm label range
print(sorted(set(data['label'])))  # should be [0, ..., num_classes - 1]
print("Total classes:", len(set(data['label'])))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [15]:
data

,Description,Rep,label
0,احمد محمد سمرى bhr.reb 94 -,bhr.rep94,129
1,مشرف بيشوي نبيل- بمناولة فادي يوسف جورج يوسف -,b.nabil@damen.com.eg,99
2,محمد سيد محمود قطب - cairo rep 176 -,cairo.rep176,170
3,Giza rep 73- فادي يوسف جورج -,giza.rep73,292
4,عبدالله مصطفي عبدالله ///giza rep 133 -,giza.rep133,276
...,...,...,...
33893,DAKREP78-احمد إبراهيم عبدالعزيز بركات ابراهيم -,dak.rep78,206
33894,IBM transfer for id IBM2409260003148 From 295...,bhr.rep102,103
33895,IBM transfer for id IBM2409260005565 From 254...,mnf.rep19,382
33896,MobileBanking fund transfer from account 2120...,alaa.ibrahim@damen.com.eg,12


In [16]:
label_counts = data['label'].value_counts()
valid_labels = label_counts[label_counts >= 2].index
data = data[data['label'].isin(valid_labels)]

In [17]:
texts = list(data['Description'])
labels = list(data['label'])

# Split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [24]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item


In [19]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-multilingual-cased",
    num_labels=559# adjust for your number of classes
)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

2025-06-19 20:50:40.405704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750366240.580909      58 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750366240.632349      58 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model.to(device)

Using device: cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [28]:
# Ensure all texts are strings (just in case there are leftover NaNs or numbers)
train_texts = [str(text) for text in train_texts]
test_texts = [str(text) for text in test_texts]


In [29]:
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=30)


In [26]:
# train_dataset = TextDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
# test_dataset = TextDataset(test_texts.tolist(), test_labels.tolist(), tokenizer)

# train_loader = DataLoader(train_dataset, batch_size=30, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=30)


AttributeError: 'list' object has no attribute 'tolist'

In [30]:
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

for epoch in range(8):  # adjust number of epochs
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")


Epoch 1 - Loss: 3150.7772
Epoch 2 - Loss: 806.6259
Epoch 3 - Loss: 335.6466
Epoch 4 - Loss: 219.3980
Epoch 5 - Loss: 174.1507
Epoch 6 - Loss: 151.6878
Epoch 7 - Loss: 129.2463
Epoch 8 - Loss: 124.9261


In [39]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model2(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {acc:.4f}")

Test Accuracy: 0.9463


In [32]:
# Save model, tokenizer, and label encoder
from transformers import BertTokenizer
import joblib
import os

output_dir = "./saved_model"
os.makedirs(output_dir, exist_ok=True)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
joblib.dump(le, f"{output_dir}/label_encoder.pkl")

print("Model, tokenizer, and label encoder saved successfully.")

Model, tokenizer, and label encoder saved successfully.


In [38]:


model2.to(device)  # use CUDA if available
model2.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [55]:
import joblib

joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']

In [36]:
import pandas as pd
from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model2(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
acc = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {acc:.4f}")

# ✅ Convert predictions and labels to original class names (if using LabelEncoder)
predicted_names = le.inverse_transform(all_preds)
actual_names = le.inverse_transform(all_labels)

# ✅ Create DataFrame
results_df = pd.DataFrame({
    "Actual": actual_names,
    "Predicted": predicted_names
})

# Show first few
print(results_df.head())

# Optional: Save to Excel
# results_df.to_excel("predictions_vs_actual.xlsx", index=False)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

## test saved model


In [35]:
model2 = BertForSequenceClassification.from_pretrained("/kaggle/working/saved_model", local_files_only=True)
tokenizer2 = BertTokenizer.from_pretrained("/kaggle/working/saved_model", local_files_only=True)
le2 = joblib.load("/kaggle/working/saved_model/label_encoder.pkl")



In [47]:


class TextDataset(Dataset):
    def __init__(self, encodings, descriptions):
        self.encodings = encodings
        self.descriptions = descriptions

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'description': self.descriptions[idx]
        }

    def __len__(self):
        return len(self.descriptions)


In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
def predict_and_evaluate(test_path, model, tokenizer, le, device, batch_size=16, save_path="predicted_vs_actual_with_flags.xlsx"):
    # Load and clean data
    test_data = pd.read_excel(test_path)
    test_data = test_data[test_data['Description'].apply(lambda x: isinstance(x, str))]
    test_data.dropna(subset=['Description', 'Rep'], inplace=True)

    # Prepare inputs
    descriptions = test_data['Description'].astype(str).tolist()
    actual_reps = test_data['Rep'].astype(str).tolist()

    encodings = tokenizer(descriptions, truncation=True, padding=True, return_tensors='pt')
    dataset = TextDataset(encodings, descriptions)
    loader = DataLoader(dataset, batch_size=batch_size)

    # Predict
    model.to(device)
    model.eval()
    all_preds, all_texts = [], []

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            texts = batch['description']

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_texts.extend(texts)

    # Decode predictions
    pred_labels = le.inverse_transform(all_preds)
    is_correct = [pred == actual for pred, actual in zip(pred_labels, actual_reps)]

    # Results DataFrame
    result_df = pd.DataFrame({
        'Description': all_texts,
        'Actual_Rep': actual_reps,
        'Predicted_Rep': pred_labels,
        'Is_Correct': is_correct
    })

    result_df.to_excel(save_path, index=False)
    return result_df


In [ ]:
result_df = predict_and_evaluate(
    test_path="your_test_file.xlsx",
    model=model,
    tokenizer=tokenizer,
    le=le,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)